# Data Preprocessing

Notebook to load raw CSVs, clean and merge them, and save a single processed file `data/processed_data.csv`.

Notes:
- Looks for raw files in `data/rawdata/`, `data/raw/`, `rawdata/`, or `archive/`.
- Uses `src.preprocessing` utilities (`clean_numeric_columns`, `parse_dates`, `save_processed`) for consistent cleaning.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Ensure project root is on sys.path so `src` imports work when running cells
p = Path.cwd()
for _ in range(6):
    if (p / 'src').exists():
        sys.path.insert(0, str(p))
        break
    p = p.parent
else:
    # fallback: add current working dir
    sys.path.insert(0, str(Path.cwd()))

# Reuse preprocessing utilities from src for consistent behavior
from src.preprocessing import clean_numeric_columns, parse_dates, save_processed

## 1. Cấu hình và Đường dẫn
Khai báo đường dẫn đến các file dữ liệu.

In [2]:
from pathlib import Path

# Enforce canonical raw directory 'data/rawdata' and required filenames
p = Path.cwd()
base_path = None
for _ in range(6):
    candidate = p / 'data' / 'rawdata'
    if candidate.exists():
        base_path = candidate
        break
    p = p.parent

if base_path is None:
    raise FileNotFoundError("Missing required raw directory 'data/rawdata' (searching upward from notebook). Please place raw files there.")

required_files = ['Orders.csv', 'Order_Details.csv', 'Customers.csv']
missing = [f for f in required_files if not (base_path / f).exists()]
if missing:
    raise FileNotFoundError(f"Missing required raw files in {base_path}: {missing}. Please place raw files into 'data/rawdata' with names: {required_files}.")

orders_path = base_path / 'Orders.csv'
details_path = base_path / 'Order_Details.csv'
customers_path = base_path / 'Customers.csv'

print(f"Using raw data directory: {base_path}")

Using raw data directory: /Users/admin/ML/3A_Superstore/data/rawdata


## 2. Đọc dữ liệu
Đọc dữ liệu từ file CSV

In [3]:
print("Loading data...")

from src.preprocessing import robust_read_processed

# If raw split files exist, read them; otherwise try loading an existing processed file
if orders_path.exists() and details_path.exists() and customers_path.exists():
    # Robust CSV reader for raw split files
    def robust_read_csv(path, **kwargs):
        try:
            return pd.read_csv(path, **kwargs)
        except Exception:
            for sep in [';', ',']:
                for enc in ['utf-8', 'latin-1']:
                    try:
                        return pd.read_csv(path, sep=sep, encoding=enc, engine='python')
                    except Exception:
                        continue
            raise

    # Load Orders
    orders = robust_read_csv(orders_path)
    print(f"Orders shape: {orders.shape}")

    # Load Details
    details = robust_read_csv(details_path)
    print(f"Details shape: {details.shape}")

    # Load Customers
    customers = robust_read_csv(customers_path)
    print(f"Customers shape: {customers.shape}")
else:
    print("Raw split files not found. Attempting to load existing processed dataset 'data/processed_data.csv'...")

    # Try to find processed_data.csv by searching upward from current working directory
    p = Path.cwd()
    processed_path = None
    for _ in range(6):
        candidate = p / 'data' / 'processed_data.csv'
        if candidate.exists():
            processed_path = str(candidate)
            break
        candidate2 = p / 'processed_data.csv'
        if candidate2.exists():
            processed_path = str(candidate2)
            break
        p = p.parent

    if processed_path is None:
        # fallback to default behavior of robust_read_processed which searches local paths
        final_df, sep = robust_read_processed()
    else:
        final_df, sep = robust_read_processed(processed_path)

    print(f"Loaded processed dataset with shape: {final_df.shape}")

Loading data...
Orders shape: (10235193, 6)
Details shape: (51185032, 7)
Customers shape: (99998, 11)


## 3. Lấy mẫu (Sampling)
Giảm kích thước dữ liệu để thuận tiện cho việc xử lý và phân tích. Chúng ta lấy mẫu theo Orders ID để đảm bảo tính toàn vẹn của đơn hàng.

In [4]:
print("Sampling orders to reduce dataset size...")

if 'final_df' in globals():
    print("Processed dataset already loaded; skipping raw-order sampling.")
else:
    # If dataset is large, sample orders to keep processing fast for prototyping
    TARGET_ORDERS = 12000
    if len(orders) > TARGET_ORDERS:
        orders = orders.sample(n=TARGET_ORDERS, random_state=42)

    print(f"Sampled Orders shape: {orders.shape}")

Sampling orders to reduce dataset size...
Sampled Orders shape: (12000, 6)


## 4. Làm sạch dữ liệu (Data Cleaning)
- Chuyển đổi định dạng tiền tệ (chuỗi có dấu phẩy sang float)
- Chuyển đổi định dạng ngày tháng
- Lọc dữ liệu chi tiết và khách hàng theo danh sách đơn hàng đã lấy mẫu

In [5]:
print("Cleaning data using src.preprocessing helpers...")

from src.preprocessing import clean_numeric_columns, parse_dates

if 'final_df' in globals():
    # Assume final_df already cleaned in previous run; ensure numeric normalization and date parsing
    final_df = clean_numeric_columns(final_df, cols=['TOTALBASKET','UNITPRICE','TOTALPRICE','AMOUNT'])
    final_df = parse_dates(final_df, col='DATE_')
    final_df = parse_dates(final_df, col='USERBIRTHDATE')
    print('Processed dataset cleaned (normalized numeric columns and parsed dates).')
else:
    # Clean Orders
    orders = clean_numeric_columns(orders, cols=['TOTALBASKET'])
    orders = parse_dates(orders, col='DATE_')

    # Filter and clean Details
    details = details[details['ORDERID'].isin(orders['ORDERID'])]
    details = clean_numeric_columns(details, cols=['UNITPRICE', 'TOTALPRICE'])

    # Filter and parse Customers
    customers = customers[customers['USERID'].isin(orders['USERID'])]
    customers = parse_dates(customers, col='USERBIRTHDATE')

    # Basic validation
required_columns = {
    'orders': ['ORDERID', 'USERID'],
    'details': ['ORDERID'],
    'customers': ['USERID']
}

for dfname, df in [('orders', orders), ('details', details), ('customers', customers)]:
    for col in required_columns[dfname]:
        if col not in df.columns:
            raise KeyError(f"Required column {col} missing in {dfname} dataframe")

    # Show concise summaries
    print('Orders:', orders.shape)
    print('Details:', details.shape)
    print('Customers:', customers.shape)

Cleaning data using src.preprocessing helpers...
Orders: (12000, 6)
Details: (59997, 7)
Customers: (11329, 11)
Orders: (12000, 6)
Details: (59997, 7)
Customers: (11329, 11)
Orders: (12000, 6)
Details: (59997, 7)
Customers: (11329, 11)


/Users/admin/ML/3A_Superstore/src/preprocessing.py:94: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')


## 5. Hợp nhất dữ liệu (Merging)
Kết nối 3 bảng Orders, Order Details, và Customers lại thành một bảng duy nhất.

In [6]:
print("Merging data...")

if 'final_df' in globals():
    print(f"Skipping merge, using existing processed data with shape: {final_df.shape}")
else:
    # Merge Orders with Details on ORDERID
    merged_orders = pd.merge(details, orders, on='ORDERID', how='left')
    
    # Merge with Customers on USERID
    # Drop overlapping columns from customers to avoid _x, _y suffixes (e.g. NAMESURNAME)
    customers_to_merge = customers.copy()
    overlap_cols = [c for c in customers_to_merge.columns if c in merged_orders.columns and c != 'USERID']
    if overlap_cols:
        print(f"Dropping overlapping columns from customers before merge: {overlap_cols}")
        customers_to_merge = customers_to_merge.drop(columns=overlap_cols)
        
    final_df = pd.merge(merged_orders, customers_to_merge, on='USERID', how='left')

    print(f"Merged shape: {final_df.shape}")

    # If extremely large (unlikely after sampling), downsample for prototyping
    if len(final_df) > 1000000:
        final_df = final_df.sample(n=1000000, random_state=42)

    print('Merged and limited (if applicable):', final_df.shape)


Merging data...
Dropping overlapping columns from customers before merge: ['NAMESURNAME']
Merged shape: (59997, 21)
Merged and limited (if applicable): (59997, 21)


## 6. Feature Engineering
Tạo các đặc trưng mới:
- **Year, Month**: Từ ngày đặt hàng
- **Age**: Tuổi của khách hàng

In [7]:
# Ensure DATE_ is datetime and create time features
final_df['Year'] = final_df['DATE_'].dt.year
final_df['Month'] = final_df['DATE_'].dt.month

# Calculate Age from USERBIRTHDATE if available
def compute_age(birthdate):
    if pd.isna(birthdate):
        return pd.NA
    try:
        return pd.Timestamp.now().year - birthdate.year
    except Exception:
        return pd.NA

final_df['Age'] = final_df['USERBIRTHDATE'].apply(compute_age)

# Quick check
print('Columns now include:', ['Year','Month','Age'])
print('Sample rows:')
final_df[['Year','Month','Age']].head()

Columns now include: ['Year', 'Month', 'Age']
Sample rows:


,Year,Month,Age
0,NaN,NaN,69
1,NaN,NaN,73
2,2021.0,1.0,35
3,NaN,NaN,30
4,NaN,NaN,33


## 7. Lưu và Kiểm tra kết quả
Lưu dữ liệu đã xử lý ra file CSV mới và hiển thị thông tin mẫu.

In [ ]:
# Remove username/surname related columns before saving
cols_to_drop = [c for c in final_df.columns if 'USERNAME' in c.upper() or 'SURNAME' in c.upper()]
if cols_to_drop:
    final_df = final_df.drop(columns=cols_to_drop, errors='ignore')

# Save processed data using utility to ensure consistent path/dirs
output_path = Path('data') / 'processed_data.csv'
save_processed(final_df, out=str(output_path))
print(f"Saved processed data to {output_path}")

# Quick inspection
print(final_df.info())
final_df.head()

Saved processed data to data/processed_data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59997 entries, 0 to 59996
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ORDERID        59997 non-null  int64         
 1   ORDERDETAILID  59997 non-null  int64         
 2   AMOUNT         59997 non-null  int64         
 3   UNITPRICE      59997 non-null  float64       
 4   TOTALPRICE     59997 non-null  float64       
 5   ITEMID         59997 non-null  int64         
 6   ITEMCODE       59997 non-null  int64         
 7   BRANCH_ID      59997 non-null  object        
 8   DATE_          23866 non-null  datetime64[ns]
 9   USERID         59997 non-null  int64         
 10  NAMESURNAME    59997 non-null  object        
 11  TOTALBASKET    59997 non-null  float64       
 12  USERNAME_      59997 non-null  object        
 13  STATUS_        59997 non-null  int64         
 14  USERGENDER     59997 n

,ORDERID,ORDERDETAILID,AMOUNT,UNITPRICE,TOTALPRICE,ITEMID,ITEMCODE,BRANCH_ID,DATE_,USERID,...,USERGENDER,USERBIRTHDATE,REGION,CITY,TOWN,DISTRICT,ADDRESSTEXT,Year,Month,Age
0,4316432,21584356,5,59.40,176.15,15973,27425,56-AN1,NaT,30892,...,M,1957-09-07,Ic Anadolu,Ankara,CANKAYA,MUTLUKENT MAH.,MUTLUKENT MAH. 1986. SOKAK 06800 CANKAYA/ANKARA,NaN,NaN,69
1,6788624,33953557,1,34.55,26.01,15934,27386,56-AN1,NaT,71967,...,M,1953-07-03,Ic Anadolu,Ankara,CANKAYA,KIRKKONAKLAR MAH.,KIRKKONAKLAR MAH. 361. SOKAK 06610 CANKAYA/A...,NaN,NaN,73
2,3377453,16887270,2,59.50,76.32,17678,28033,56-AN4,2021-01-04,55698,...,F,1991-02-13,Ic Anadolu,Ankara,KECIOREN,GUCLUKAYA MAH.,GUCLUKAYA MAH. KEREM SOKAK 06310 KECIOREN/AN...,2021.0,1.0,35
3,6141661,30714643,8,32.45,242.40,7449,22972,56-AN1,NaT,19596,...,F,1996-11-12,Ic Anadolu,Ankara,MAMAK,EGE MAH.,EGE MAH. 758. SOKAK 06480 MAMAK/ANKARA,NaN,NaN,30
4,3519105,17594071,7,64.50,392.70,22967,44770,56-AN4,NaT,37719,...,F,1993-09-04,Ic Anadolu,Ankara,ETIMESGUT,BAGLICA MAH.,BAGLICA MAH. 1000. SOKAK 06790 ETIMESGUT/ANKARA,NaN,NaN,33


In [9]:
# Re-save processed data using safe save (sanitize + explicit sep/encoding)
import importlib
import src.preprocessing as sp
importlib.reload(sp)
sp.save_processed(final_df, out='data/processed_data.csv', sep=';')
print('Re-saved processed_data.csv using save_processed')

Re-saved processed_data.csv using save_processed
